In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import cond

In [3]:
np.random.seed(0) 

X = np.random.randn(200, 20)   
beta_true = np.random.randn(20) 
noise = 0.1 * np.random.randn(200) 
y = X @ beta_true + noise       

print("Condition number of X:", cond(X))
print("First 5 values of y:", y[:5])


Condition number of X: 1.703051197787162
First 5 values of y: [-2.4307426  -5.49401837  0.93889084 -2.7802066  -8.99570186]
